In [1]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

In [2]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

# set kinetic parameters
with open("rate_parameters.json") as infile:
    jsdata = json.load(infile)

params = jsdata["kin1"]

Copyright **Peter Kraus and Paolo Raiteri**, January 2021

## Numerical solution of chemical equilibrium problems #1
Imagine a simple dimerisation reaction
\begin{equation}
2A \to B
\end{equation}

whose equilibrium constant can be writen as
\begin{equation}
K_{eq} = \frac{[B]}{[A]^2} = 0.156
\end{equation}

ALthough this is a simple problem that can be solved analytically, in this workshop we will learn how we can use an iterative method to numerically solve it.
We will use a self consisten procedure, which can be applied to a large number of problems, for which it is not possible or it is too complicated to get an analytic solution.

Imagine to mix the reagents and then to be able to monitor at discrete and equally spaced time intervals (*timesteps*) the concentration of all the species in the system. What you will see is that the concetrations will change and tendo to the equilibrium value. As you have learnt in first year, the reation quotient, $Q$, can be used to decided which way to reaction will proceed, and that at equilibruim the reaction quotient is equal to the equilibrium constant. Hence, the reation quotient can be seen as the *driving force* that brings the system towards equilibrium.

Analogously to the procedure you would use to analytically solve this problem, we will start from the *ICE* table

|      | [A]        | [B]
| :--- | :--------: |:---------:
| *I*  | [A]$_0$    | [B]$_0$
| *C*  | -2x        | x
| *E*  | [A]$_0$-2x | [B]$_0$+x

Here below you can see the working principle of the self-consistent procesure that we will emply

1. compute the reaction quotient at beginning of the experiment
\begin{equation}
Q = \dfrac{\mathrm{[B]}_0}{\mathrm{[A]}^2_0}
\end{equation}

2. determine the driving force.
You have learnt that if $Q/K_{eq}>1$ the reaction proceeds towards the reactants, which we can call the *negative* direction. On the other hand, if $Q/K_{eq}<1$ it goes towards the products, *i.e.* the positive direction.
This means that the *driving force* has opposite sign when the $Q/K_{eq}$ becomes smaller or larger than one.
A common mathematical function that has the same property is the logarithm, which indeed can provide a way to define the *force force* that has the desired sign using the $Q/K_{eq}$ ratio
\begin{equation}
F \propto -\ln\bigg[\frac{Q}{K_{eq}}\bigg]
\end{equation}

3. After an arbitrarily short time interval has passed you can imagine that the concentrations of all the species in the system have changed according the the *C* row of the *ICE* table by an amount that is prortional to the driving force, *i.e*
\begin{equation}
x = -\delta\ln\bigg[\frac{Q}{K_{eq}}\bigg]
\end{equation}

4. Repeat from step 1 until there are no more change in the concentrations of all the species.

Follow now the demstrator explaining how to create an excel spreadsheet that implements those steps.

Now try yourself.

- Click `Download CSV` to export the data as a CSV file to verify your result.


In [3]:
def initialise():
    global nPoints
    global concA, concB
    global Keq
    global delta

    nPoints = 20
    concA = 1
    concB = 0.1
    Keq = 0.156
    delta = 0.001

def addLine(t,x,y,res,Q):
    var_list = []
    var_list.append(t)
    var_list.append(x)
    var_list.append(y)
    var_list.append(Q)
    res.loc[len(res)] = var_list

initialise()


In [4]:
out_P = ipw.Output()
out_L = ipw.Output()

with out_L:
    display(Markdown("[Download CSV](./results.csv)"))

    
def force(Q,k):
    if (abs(Q) > 1.e-6):
        force = - math.log(Q/k)
    else:
        force = 1.

    return force

def calc1(btn):
    out_P.clear_output()
    
    if os.path.exists(os.path.join(os.getcwd(), "results.csv")):
        os.remove(os.path.join(os.getcwd(), "results.csv"))
    res = pd.DataFrame(columns=["step" , "[A]" , "[B]", "Q"])

    A = float(concA_text.value)
    B = float(concB_text.value)
    dx = float(delta_text.value)
    k = float(Keq_text.value)
    n = int(nPoints_text.value)

    Q = B / math.pow(A,2)
    addLine(0,A,B,res,Q)

    for i in range(0, n):
        f = force(Q,k)
        A = A - 2 * dx * f
        B = B + dx * f
        
        Q = B / math.pow(A,2)
        addLine(i,A,B,res,Q)
        # Append result

        
    res.to_csv("results.csv", index=False)
    with out_P:
        display(res.tail(n))

btn_calc1 = ipw.Button(description="Get Data", layout=ipw.Layout(width="150px"))
btn_calc1.on_click(calc1)

rows = []

# Equilibrium constant
Keq_text = ipw.Text(str(Keq))

# Initial concentrations
concA_text = ipw.Text(str(concA))

concB_text = ipw.Text(str(concB))

# delta concentration
delta_text = ipw.Text(str(delta))


# Nmber of data points
nPoints_text = ipw.Text(str(nPoints))

rows.append(ipw.HBox([ipw.Label('Initial concentration of A   :  '),concA_text]))
rows.append(ipw.HBox([ipw.Label('Initial concentration of B   :  '),concB_text]))
rows.append(ipw.HBox([ipw.Label('Equilibrium constant         :  '),Keq_text]))
rows.append(ipw.HBox([ipw.Label('Delta concentration          :  '),delta_text]))
rows.append(ipw.HBox([ipw.Label('Number of data point required:  '),nPoints_text]))

rows.append(ipw.HBox([btn_calc1]))

rows.append(ipw.HBox([out_L]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)